## Quantum Walks

Quantum walks are quantum analogs of classical random walks. In contrast to the classical random walk, where the walker occupies definite states and the randomness arises due to stochastic transitions between states, in quantum walks randomness arises through quantum superposition of states, non-random, reversible unitary evolution and collapse of the wave function due to state measurements.

Below we write a code for implementing quantum walk on a line with 16 nodes

In [82]:
from classiq import *
import classiq

In [83]:
classiq.authenticate()

/home/hitanshugedam/anaconda3/lib/python3.11/site-packages/classiq/_internals/authentication/token_manager.py:99: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


In [84]:
from classiq.execution import ExecutionPreferences

In [85]:
size = 4 #log_2(16) -> log16 with base 2 

In [86]:
num_nodes = 16 #no. of nodes on the line

In [87]:
@qfunc
def prepare_minus(x: QBit):
    X(x)
    H(x)

@qfunc
def diffuser_oracle(aux: Output[QNum], x:QBit):
    aux ^= (x!=0)

@qfunc
def zero_diffuser(x: QNum):
    aux = QNum('aux')
    allocate(1,aux)
    within_apply(compute=lambda: prepare_minus(aux),
              action=lambda: diffuser_oracle)

In [88]:
def C_iterate(i: CInt, vertices: QNum, adjacent_vertices: QNum): #C iteration for the scope of the assignment i.e. Line with 16 nodes
    probs = [0] * num_nodes
    if i == 0:
        probs[i+1] = 1.0 #probability of moving right from node 0
    elif i == num_nodes - 1:
        probs[i-1] = 1.0 #probability of moving left from node 15
    else:
        probs[i+1] = 0.5
        probs[i-1] = 0.5
    print(f'Node={i}, prob vec ={probs}')
    control(ctrl = vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=probs, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuser(adjacent_vertices)))


In [89]:
@qfunc 
def C_operator(vertices:QNum, adjacent_vertices: QNum): #C operator as defined in the paper for quantum walks
    for i in range(num_nodes):
        C_iterate(i,vertices,adjacent_vertices)


In [90]:
@qfunc
def edge_oracle(res:Output[QBit], vertices: QNum, adjacent_vertices: QNum): #checking for connected vertices
    res |= (((vertices - adjacent_vertices) == 1) | ((vertices - adjacent_vertices) == -1)) #checking for adjacent nodes on the line
@qfunc 
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]): #applying the SWAP 
    repeat(count= x.len,
           iteration= lambda i: SWAP(x[i],y[i])) 

@qfunc 
def S_operator(vertices:QNum, adjacent_vertices: QNum):
    res = QNum('res')
    edge_oracle(res,vertices,adjacent_vertices)
    control(ctrl= res==1,
        operand= lambda: bitwise_swap(vertices,adjacent_vertices))

We have now defined the C operator and the S operator for our quantum program. Now let's put both of them together in our main function

In [91]:
@qfunc 
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):
    allocate(size, vertices)  # Allocate qubits for vertices
    hadamard_transform(vertices)  # Apply Hadamard transform to initialize superposition
    allocate(size, adjacent_vertices)  # Allocate qubits for adjacent vertices
    
    C_operator(vertices, adjacent_vertices)  # Applying the C operator
    S_operator(vertices, adjacent_vertices)  # Applying the S operator


In [92]:
# Create and synthesize the quantum model
quantum_model = create_model(main)
qprog = synthesize(quantum_model)
show(qprog)

Node=0, prob vec =[0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=1, prob vec =[0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=2, prob vec =[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=3, prob vec =[0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=4, prob vec =[0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=5, prob vec =[0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=6, prob vec =[0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
Node=7, prob vec =[0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
Node=8, prob vec =[0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
Node=9, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
Node=10, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
Node=11, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
Node=12, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0]
Node=13, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0]
Node

![image](/home/hitanshugedam/Downloads/pic1.png)